In [107]:
import pandas as pd
from datetime import date, timedelta
import cbpro
import numpy as np
import matplotlib.pyplot as plt

from portfolio import Asset
USD = Asset('USD')
date_list = pd.date_range(start = date.fromisoformat('2000-01-01'), end = date.today(), freq=timedelta(days=1)).to_pydatetime().tolist()
USD_2000 = pd.DataFrame({'date_time': date_list,
                    'low':[1] * len(date_list),
                    'high':[1] * len(date_list),
                    'open':[1] * len(date_list),
                    'close':[1] * len(date_list),
                    'volume':[1000]* len(date_list)})
USD.update_history(USD_2000)

In [2]:
with open('CoinBaseAPIKey.key', 'r') as f:
    contents = f.readlines()
cb_key = contents[0].split(':')[-1].strip()
cb_secret = contents[1].split(':')[-1].strip()

In [3]:
#Download data
public_client = cbpro.PublicClient()
slots = [('2023-01-01', '2023-06-01'), ('2022-07-01', '2022-12-31'), ('2022-01-01', '2022-06-30'), ('2023-06-02', '2023-12-01')]
BTC = Asset('BTC')
ETH = Asset('ETH')
USDT = Asset('USDT')

for slot in slots:
    BTC_USD = public_client.get_product_historic_rates('BTC-USD', slot[0], slot[1], 86400)
    ETH_USD = public_client.get_product_historic_rates('ETH-USD', slot[0], slot[1], 86400)
    USDT_USD = public_client.get_product_historic_rates('USDT-USD', slot[0], slot[1], 86400)
    btc_data= pd.DataFrame(BTC_USD)
    btc_data.columns = [ 'date_time', 'low', 'high', 'open', 'close', 'volume' ]
    btc_data['date_time']= btc_data['date_time'].apply(lambda x: date.fromtimestamp(x))
    BTC.update_history(btc_data)

    ETH_data= pd.DataFrame(ETH_USD)
    ETH_data.columns = [ 'date_time', 'low', 'high', 'open', 'close', 'volume' ]
    ETH_data['date_time']= ETH_data['date_time'].apply(lambda x: date.fromtimestamp(x))
    ETH.update_history(ETH_data)

    USDT_data= pd.DataFrame(USDT_USD)
    USDT_data.columns = [ 'date_time', 'low', 'high', 'open', 'close', 'volume' ]
    USDT_data['date_time']= USDT_data['date_time'].apply(lambda x: date.fromtimestamp(x))
    USDT.update_history(USDT_data)

In [86]:
class Portfolio:
    def __init__(self, origination_date = date.fromisoformat('2000-01-01'), initial_deposit = 0):
        idx  = pd.MultiIndex(levels=[[],[]],
                          codes=[[],[]],
                          names=[u'date_time', u'ticker'])
        my_columns = [u'change', u'note']
        self.transactions  = pd.DataFrame(index=idx, columns=my_columns)
        self.value = pd.DataFrame(columns = ['value'], index = pd.Index([], name='date_time'))
        self.orig_date = origination_date
        self.update_transactions(transaction_date = origination_date,
                                ticker = 'USD',
                                qty = initial_deposit,
                                note = 'Initial deposit')


    def update_transactions(self, ticker:str, qty:float, transaction_date = date.today(), note = ''):
        self.transactions.loc[(transaction_date,ticker),:] = [qty, note]

    def get_positions(self, on_date = date.today()):
        '''
        Returns portfolio composition on the particular date
        '''
        # First, get the mask for all records before the date
        time_mask= self.transactions.index.get_level_values('date_time')<=on_date
        # Filter the df with this mask, group by ticker
        positions = self.transactions[time_mask]['change'].groupby(level = 'ticker').sum()
        # Convert resulting Series to DataFrame
        positions = pd.DataFrame(positions, index=pd.Index(positions.index, name = 'ticker'))
        positions.columns = ['position_size']
        
        # Add the value in USD by multiplying on the asset price at this date
        positions['position_value'] = positions.index.to_series().apply(
                                                lambda x: Asset.asset_dict[str(x)].price_on_date(on_date)
                                                )
        positions['position_value'] = positions['position_value']*positions['position_size']
        return positions

    def get_value(self, on_date = date.today()):
        self.update_value()
        return self.value.loc[on_date]
    
    def update_value(self):
        #self.value.drop(self.value.index, inplace=True)
        date_to_add = date.fromisoformat('2023-11-30')
        while date_to_add >= self.orig_date:
            composition_at_date = self.get_positions(date_to_add).dropna()
            value_to_add = composition_at_date.position_value.sum()
            self.value.loc[date_to_add] = value_to_add
            date_to_add = date_to_add - timedelta(days=1)

In [91]:
my_portfolio = Portfolio(origination_date=date.fromisoformat('2021-12-01'), initial_deposit=100)
my_portfolio.update_transactions(ticker='BTC', qty=1, transaction_date=date.fromisoformat('2022-01-01'))
my_portfolio.update_transactions(ticker='USDT', qty=40000, transaction_date=date.fromisoformat('2022-01-01'))
my_portfolio.update_transactions(ticker='BTC', qty=-0.5, transaction_date=date.fromisoformat('2022-01-02'))
my_portfolio.update_transactions(ticker='USDT', qty=-10000, transaction_date=date.fromisoformat('2022-01-02'))
my_portfolio.update_transactions(ticker='BTC', qty=-0.25, transaction_date=date.fromisoformat('2022-01-10'))
my_portfolio.update_transactions(ticker='USDT', qty=-20000, transaction_date=date.fromisoformat('2022-01-10'))
my_portfolio.transactions

change             note
date_time  ticker                         
2021-12-01 USD        100  Initial deposit
2022-01-01 BTC          1                 
           USDT     40000                 
2022-01-02 BTC       -0.5                 
           USDT    -10000                 
2022-01-10 BTC      -0.25                 
           USDT    -20000